In [1]:
from ib_insync import *
import numpy as np
import pandas as pd
from datetime import datetime
from trading_stratigies import write_file, sort_first_by_second
import matplotlib as plt

In [2]:
# The function to get contract historical data
def get_contract_data(contract, look_back, resolution, side, end_time):
    bar = ib.reqHistoricalData(contract, endDateTime = end_time, durationStr=look_back,
        barSizeSetting= resolution, whatToShow=side, useRTH=True, formatDate=1)
# use util in ib-insync to convert bar data into pandas dataframe    
    df = util.df(bar)
    return df 

# The function to get stock historical data
def get_stock_data(ticker, look_back, resolution, side, end_time):
    contract = Contract(symbol = ticker, secType = 'STK', exchange = 'SMART',currency = 'USD')   
    if len(ib.qualifyContracts(contract)) == 0:
        return pd.DataFrame()
    return get_contract_data(contract, look_back, resolution, side, end_time)

# The function to get stock return historical data
def get_return_history(ticker_lst, look_back, resolution, side, end_time):
    column_names = np.array(get_stock_data("ABC", look_back, resolution, side, end_time)['date'][1:])
    Stocks_return_history_tmp = pd.DataFrame(columns = column_names)
    for ticker_index in range(len(ticker_lst)):
        ticker = ticker_lst [ticker_index]
        stock_data_tmp = get_stock_data(ticker, look_back, resolution, side, end_time)
        if type(stock_data_tmp) != type(pd.DataFrame()) or not "close" in stock_data_tmp:
            print ("no history available for ticker " + ticker)
            continue 
        if len(stock_data_tmp) - 1 < len(column_names):
            print ("not long enough history for " + str(ticker))
        elif len(stock_data_tmp) - 1 > len(column_names):
            print ("excessive " + str(len(stock_data_tmp)-len(column_names)) + 
                   " trading days for " + str(ticker))
        else:
            equity_close_history_tmp = stock_data_tmp.close
            Stocks_return_history_tmp.loc[ticker_lst[ticker_index]] = np.diff(np.log(equity_close_history_tmp))
    return Stocks_return_history_tmp

In [3]:
# Connect to IB Server
ib=IB()
util.startLoop()
ib.connect('127.0.0.1',7497, clientId= 11)

<IB connected to 127.0.0.1:7497 clientId=11>

In [4]:
def linear_regress_history(x,y):
    from sklearn.linear_model import LinearRegression
    x, y = np.array(x), np.array(y)
    X = x.reshape((-1, 1))
    Y = y
    model = LinearRegression().fit(X, Y)
    intercept = model.intercept_
    slope = model.coef_[0]
    res = y - slope*x - intercept
    sigma = np.sqrt(np.mean(res**2))
    zscore = res/sigma
#     print(zscore)
    return [slope, intercept, sigma]

In [5]:
def initialize_back_test_pair (pair, look_back, num_strategy_look_back, resolution, start_time, end_time):
    df_0_bid = get_stock_data(pair[0], look_back, resolution, "Bid", end_time)
    df_0_ask = get_stock_data(pair[0], look_back, resolution, "Ask", end_time)
    df_0_trades = get_stock_data(pair[0], look_back, resolution, "Trades", end_time)
    df_1_bid = get_stock_data(pair[1], look_back, resolution, "Bid", end_time)
    df_1_ask = get_stock_data(pair[1], look_back, resolution, "Ask", end_time)
    df_1_trades = get_stock_data(pair[1], look_back, resolution, "Trades", end_time)
    if df_0_bid is None or df_1_bid is None or len(df_0_bid) == 0 or len(df_1_bid) ==0:
        if_success = False
        [history_price, history_return, history_operation, history_log] = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]
        print("Proper history is not available for stock pair " + str(pair) + "\n")
        return [history_price, history_operation, history_log, if_success]
    if df_0_bid.date.equals(df_1_bid.date):
        history_price = pd.DataFrame({pair[0]+" Bid Open": df_0_bid.open, pair[0]+" Bid Close": df_0_bid.close, 
                                    pair[0]+" Ask Open": df_0_ask.open, pair[0]+" Ask Close": df_0_ask.close,
                                    pair[0]+" Trades Open": df_0_trades.open, pair[0]+" Trades Close": df_0_trades.close, 
                                    pair[0]+" Log Trades Open": np.log(df_0_trades.open), pair[0]+" Log Trades Close": np.log(df_0_trades.close), 
                                    pair[1]+" Bid Open": df_1_bid.open, pair[1]+" Bid Close": df_1_bid.close, 
                                    pair[1]+" Trades Open": df_1_trades.open, pair[1]+" Trades Close": df_1_trades.close,
                                    pair[1]+" Ask Open": df_1_ask.open, pair[1]+" Ask Close": df_1_ask.close,
                                    pair[1]+" Log Trades Open": np.log(df_1_trades.open), pair[1]+" Log Trades Close": np.log(df_1_trades.close)})
        #history_all.index = [datetime.strptime(str(date), '%Y-%m-%d') for date in df_0_bid.date]
        history_price.index = df_0_bid.date
        history_price = history_price.iloc [list(history_price.index).index(start_time) - num_strategy_look_back:, :]
        
        #history_return = get_return_history(pair, look_back, resolution, "Trades", end_time).T
        #history_return = history_return.iloc [list(history_return.index).index(start_time) - num_strategy_look_back:, :]
        if_success = True
        operation_index = history_price.index [(history_price.index >= start_time) * (history_price.index <= end_time)]
        columns_name = [pair[0]+" Shares (Open)", pair[1]+" Shares (Open)", pair[0]+" Shares (Close)", pair[1]+" Shares (Close)", \
                  "Hedge Ratio", "Zscore (Open)", "Zscore (Close)","Pair Operation", "If Stop Loss", \
                  pair[0]+" Operation on Strategy", pair[1]+" Operation on Strategy", \
                  "Leverage (Open)", "Leverage (Close)", "If Margin Call (Close)",\
                  pair[0]+ " Operation on Margin Call (Close)", pair[1]+ " Operation on Margin Call (Close)", \
                  "Portfolio Value (Open)", "Portfolio Value (Close)", "Cash (Open)", "Cash (Close)", \
                  "Holdings Value (Open)", "Holdings Value (Close)", "Net Asset (Open)" ,"Net Asset (Close)", \
                  "Friction", "Day PnL", "Night PnL", "Daily PnL", "Cumulative PnL", "Daily Return", "Cumulative Return"]
        history_operation = pd.DataFrame(None, index = operation_index, columns = columns_name)
        x_array = history_price.loc[history_operation.index[0]:, pair[0]+" Log Trades Close"]
        y_array = history_price.loc[history_operation.index[0]:, pair[1]+" Log Trades Close"]
        [slope, intercept, sigma] = linear_regress_history(x_array, y_array)
        history_price["Zscore"] = (y_array - slope*x_array - intercept)/sigma
        history_log = pd.DataFrame(None, index = operation_index, columns = ["Type", "Message"])
    else:
        print("Proper history is not available for stock pair " + str(pair) + "\n")
        if_success = False
        [history_price, history_return, history_operation, history_log] = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]
    return [history_price, history_operation, history_log, if_success]

Peer closed connection
